## Projeto 1

In [1]:
pip install --upgrade imutils

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'd:\Users\heinr\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [3]:

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import re
import random

In [4]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import re
import random

from tensorflow.keras.applications import ResNet50, InceptionV3, InceptionResNetV2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import preprocess_input
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler  

from PIL import Image

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score, classification_report
from sklearn.preprocessing import Normalizer, MinMaxScaler, StandardScaler, RobustScaler, QuantileTransformer, LabelEncoder

Preprocessamento

In [5]:
def crop_brain_contour(image, plot=False):
    
    import imutils
    import cv2
    from matplotlib import pyplot as plt
    
    # Convert the image to grayscale, and blur it slightly
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

     #Threshold the image, then perform a series of erosions +
     #dilations to remove any small regions of noise
    thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=2)

    #Find contours in thresholded image, then grab the largest one
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    c = max(cnts, key=cv2.contourArea)
    

    # Find the extreme points
    extLeft = tuple(c[c[:, :, 0].argmin()][0])
    extRight = tuple(c[c[:, :, 0].argmax()][0])
    extTop = tuple(c[c[:, :, 1].argmin()][0])
    extBot = tuple(c[c[:, :, 1].argmax()][0])
    new_image = image[extTop[1]:extBot[1], extLeft[0]:extRight[0]]            
    
    return new_image

In [6]:
def corta_img(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY, 0.7)
    #plt.imshow(gray)

    (T, thresh) = cv2.threshold(gray, 155, 255, cv2.THRESH_BINARY)
    plt.imshow(thresh)
    # (T, threshInv) = cv2.threshold(gray, 155, 255, cv2.THRESH_BINARY_INV)
    # plt.imshow(threshInv)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (10, 5))
    closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    closed = cv2.erode(closed, None, iterations = 19)
    closed = cv2.dilate(closed, None, iterations = 17)
    #plt.imshow(closed)
    ret,mask = cv2.threshold(closed, 155, 255, cv2.THRESH_BINARY) 
    #apply AND operation on image and mask generated by thrresholding
    final = cv2.bitwise_and(image,image,mask = mask) 

    return final

In [7]:
def equalizeHist(img):
    b,g,r = cv2.split(img)
    
    #apply hist equ on the three channels separately
    b = cv2.equalizeHist(b)
    g = cv2.equalizeHist(g)
    r = cv2.equalizeHist(r)
    
    #merge all the three channels
    equ = cv2.merge((b,g,r))
    
    #convert it to RGB to visualize
    new_img = cv2.cvtColor(equ,cv2.COLOR_BGR2RGB);
    return new_img

In [7]:
def clahe(img):
    #do the same as we did for histogram equalization
    #set the clip value and the gridsize changing these values will give different output
    clahe = cv2.createCLAHE(clipLimit=6, \
                        tileGridSize=(16,16))
    
    #split the three channels
    b,g,r = cv2.split(img)
    
    #apply CLAHE on the three channels separately
    b = clahe.apply(b)
    g = clahe.apply(g)
    r = clahe.apply(r)
    
    #merge the three channels
    bgr = cv2.merge((b,g,r))
    
    #convert to RGB and plot
    newimage = cv2.cvtColor(bgr,cv2.COLOR_BGR2RGB);
    return newimage

In [8]:
def detecta_anorm(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY, 0.7)
    #plt.imshow(gray)

    (T, thresh) = cv2.threshold(gray, 155, 255, cv2.THRESH_BINARY)
    #plt.imshow(thresh)
    # (T, threshInv) = cv2.threshold(gray, 155, 255, cv2.THRESH_BINARY_INV)
    # plt.imshow(threshInv)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (10, 5))
    closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    closed = cv2.erode(closed, None, iterations = 19)
    closed = cv2.dilate(closed, None, iterations = 17)
    #plt.imshow(closed)

    def auto_canny(image, sigma=0.33):
        # compute the median of the single channel pixel intensities
        v = np.median(image)
        # apply automatic Canny edge detection using the computed median
        lower = int(max(0, (1.0 - sigma) * v))
        upper = int(min(255, (1.0 + sigma) * v))
        edged = cv2.Canny(image, lower, upper)
        # return the edged image
        return edged
    canny = auto_canny(closed)

    (cnts, _) = cv2.findContours(canny.copy(), cv2.RETR_EXTERNAL,
    cv2.CHAIN_APPROX_SIMPLE)
    new_img = cv2.drawContours(image, cnts, -1, (0, 0, 255), 2)
    return new_img

### -------------------------- COMEÇA AQUI ------------------------------

## Features Treinamento:

In [9]:
#
# Carrega as imagens de treinamento e salva os labels
#

train_path = ('train/')      
dirs = os.listdir( train_path )
img_size= 224

train_img = []
indices = []

for file in dirs:
   numbers = re.sub('[^0-9]', '', file)
   indices.append(numbers)

indices = (np.array(indices))
indices = sorted(indices, key=int)

for i in range(len(indices)):
   img_ori = cv2.imread(train_path + str(indices[i])+'.jpg')
   img_crop = crop_brain_contour(img_ori, True)
   #img_clahe = clahe(img_crop)
   img_equalize = equalizeHist(img_crop)
   #img_anorm = detecta_anorm(img_equalize)
   #img_anorm = corta_img(img_equalize) 
   
   train_img.append(cv2.resize(img_equalize, (img_size,img_size)))  

train_img = (np.array(train_img))

In [10]:
#
# Carrega o modelo ResNet50
#

base_model = ResNet50(weights='imagenet')
model = Model(inputs=base_model.input, outputs=base_model.get_layer('avg_pool').output)

In [ ]:
#
# extrator de features train
#

train_features_array = np.zeros((train_img.shape[0],2048))

for i, img_pos in enumerate(train_img):
  try:
    img = img_pos
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = model.predict(x)
    features = features.reshape(2048,)
    train_features_array[i,:] = features
  except:
    print(img_pos)

print(train_features_array.shape)

compress = pd.DataFrame(train_features_array)  
compress.to_csv('./train_features_crop_equalize.csv', index=False)

## Features Testes:

In [ ]:
test_path = ('./test/')
dirs = os.listdir( test_path )
dirs.sort()
img_size= 224

test_img = []
test_label = []

for file in dirs:
    if ".jpg" in file:
        img_ori = cv2.imread(test_path + str(file))
        img_crop = crop_brain_contour(img_ori, True)
        #img_clahe = clahe(img_crop)
        img_equalize = equalizeHist(img_crop)
        #img_anorm = detecta_anorm(img_equalize) 
        #img_anorm = corta_img(img_equalize) 
        test_img.append(cv2.resize(img_equalize, (img_size,img_size)))
        test_label.append(file.split('.')[0])

test_img = (np.array(test_img))
print(test_label)

In [ ]:
#
# Gerador de features test
#

teste_features_array = np.zeros((test_img.shape[0],2048))

for i, img_pos in enumerate(test_img):
  try:
    img = img_pos
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = model.predict(x)
    features = features.reshape(2048,)
    teste_features_array[i,:] = features
  except:
    print(img_pos)

compress = pd.DataFrame(teste_features_array)  
compress.to_csv('./test_features_crop_equalize.csv', index=False)